In [1]:
pip install transformers

  Using cached transformers-4.39.0-py3-none-any.whl.metadata (134 kB)
  Using cached huggingface_hub-0.21.4-py3-none-any.whl.metadata (13 kB)
  Using cached regex-2023.12.25-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (40 kB)
  Using cached tokenizers-0.15.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
  Using cached safetensors-0.4.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.8 kB)
  Using cached fsspec-2024.3.1-py3-none-any.whl.metadata (6.8 kB)
Using cached transformers-4.39.0-py3-none-any.whl (8.8 MB)
Using cached huggingface_hub-0.21.4-py3-none-any.whl (346 kB)
Using cached regex-2023.12.25-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (785 kB)
Using cached safetensors-0.4.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.3 MB)
Using cached tokenizers-0.15.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.6 MB)
Using cached fsspec-2024.3.1-py3-none-any.whl (

In [2]:
pip install scikit-learn

  Using cached scikit_learn-1.4.1.post1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (11 kB)
  Using cached scipy-1.12.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (60 kB)
  Using cached joblib-1.3.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.4.0-py3-none-any.whl.metadata (13 kB)
Using cached scikit_learn-1.4.1.post1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.1 MB)
Using cached joblib-1.3.2-py3-none-any.whl (302 kB)
Using cached scipy-1.12.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (38.4 MB)
Using cached threadpoolctl-3.4.0-py3-none-any.whl (17 kB)
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install transformers[torch]

  Using cached accelerate-0.28.0-py3-none-any.whl.metadata (18 kB)
Using cached accelerate-0.28.0-py3-none-any.whl (290 kB)
Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install accelerate -U

Note: you may need to restart the kernel to use updated packages.


In [8]:
import pandas as pd
import numpy as np

In [9]:
merged_data = pd.read_csv('merged_data_csv.csv')

In [10]:
display(merged_data)

,article,label
0,The government has deployed financial incentiv...,real
1,told us why she would not vote for vote with m...,real
2,s Bulletin of Great returns the ball during a ...,fake
3,If you have ever wondered whats the best busin...,fake
4,Institute of Technology Professor and architec...,fake
...,...,...
21940,Nuclear weapons In is not about to go to war w...,fake
21941,Authors will join hundreds of protesters in th...,real
21942,During my maternity leave I had a conversation...,real
21943,Nowhere have the tentacles of the blob made mo...,real


In [12]:
from transformers import Trainer, TrainingArguments, AutoModelForSequenceClassification, AutoTokenizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import torch
#from datasets import Dataset
from torch.utils.data import Dataset, DataLoader

# Define model name and load model
model_name = "distilgpt2"
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
#tokenizer.padding_side = "right"  # set "left" or "right" depending on tokenizer
#tokenizer.add_special_tokens({'pad_token': tokenizer.eos_token})

# Ensure padding token is defined in the model configuration
if model.config.pad_token_id is None:
    model.config.pad_token_id = tokenizer.pad_token_id  # Set pad token ID in model configuration

# resize model embedding to match new tokenizer
model.resize_token_embeddings(len(tokenizer))

# Split data into train, validation, and test sets
train_texts, test_texts, train_labels, test_labels = train_test_split(merged_data['article'], merged_data['label'], test_size=0.2, random_state=42)
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=0.2, random_state=42)

# Encode labels
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
val_labels_encoded = label_encoder.transform(val_labels)

# Tokenize data
train_encodings = tokenizer(list(train_texts), truncation=True, padding=True)
val_encodings = tokenizer(list(val_texts), truncation=True, padding=True)
test_encodings = tokenizer(list(test_texts), truncation=True, padding=True)

# Define PyTorch Dataset class
class TextClassificationDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

# Create Dataset objects
train_dataset = TextClassificationDataset(train_encodings, train_labels_encoded)
val_dataset = TextClassificationDataset(val_encodings, val_labels_encoded)

# Fine-tuning model
training_args = TrainingArguments(
    output_dir="./output",
    per_device_train_batch_size=4,
    num_train_epochs=3,
    logging_dir='./logs',
)

# Define Trainer and start training
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

trainer.train()

# Evaluating model
eval_result = trainer.evaluate(eval_dataset=val_dataset)
print(eval_result)

# Access the model's parameters
model_parameters = model.parameters()

# Iterating through parameters and printing the values
for param in model_parameters:
    print(param)

# Tokenize the test data
test_encodings = tokenizer(list(test_texts), truncation=True, padding=True)

# Create test dataset object
test_dataset = TextClassificationDataset(test_encodings, label_encoder.transform(test_labels))

# Evaluating model on test dataset
test_result = trainer.evaluate(eval_dataset=test_dataset)
print(test_result)


Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at distilgpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/conda/lib/python3.11/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Step,Training Loss
500,0.461800
1000,0.364100
1500,0.286500
2000,0.270800
2500,0.255200
3000,0.259200
3500,0.228200
4000,0.155100
4500,0.115100
5000,0.094400


{'eval_loss': 0.1663936823606491, 'eval_runtime': 48.1754, 'eval_samples_per_second': 72.9, 'eval_steps_per_second': 9.113, 'epoch': 3.0}
Parameter containing:
tensor([[-0.1445, -0.0455,  0.0042,  ..., -0.1523,  0.0184,  0.0991],
        [ 0.0573, -0.0722,  0.0234,  ...,  0.0603, -0.0042,  0.0478],
        [-0.1106,  0.0386,  0.1948,  ...,  0.0421, -0.1141, -0.1455],
        ...,
        [-0.0713, -0.0164,  0.0183,  ...,  0.0815,  0.1330, -0.0734],
        [ 0.1963,  0.0206,  0.0171,  ..., -0.0816,  0.0744, -0.0292],
        [ 0.0342,  0.0640,  0.0305,  ...,  0.0291,  0.0942,  0.0639]],
       device='cuda:0', requires_grad=True)
Parameter containing:
tensor([[-0.0205, -0.1992,  0.0017,  ..., -0.0479,  0.0337,  0.0529],
        [ 0.0261, -0.0576, -0.0955,  ...,  0.0311,  0.0138,  0.0036],
        [ 0.0048, -0.0872,  0.0490,  ...,  0.0205,  0.0241, -0.0218],
        ...,
        [ 0.0018,  0.0037, -0.0567,  ...,  0.0163, -0.0064,  0.0004],
        [ 0.0023, -0.0021, -0.0490,  ...,  0.02

In [13]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Evaluate the model on test dataset
test_result = trainer.predict(test_dataset)

# Get predicted labels
predicted_labels = test_result.predictions.argmax(axis=1)

# Get true labels
true_labels = test_dataset.labels

# Compute evaluation metrics
accuracy = accuracy_score(true_labels, predicted_labels)
precision = precision_score(true_labels, predicted_labels)
recall = recall_score(true_labels, predicted_labels)
f1 = f1_score(true_labels, predicted_labels)

print(f"Test Accuracy: {accuracy}")
print(f"Test Precision: {precision}")
print(f"Test Recall: {recall}")
print(f"Test F1 Score: {f1}")

Test Accuracy: 0.9801777170198223
Test Precision: 0.9877363311190598
Test Recall: 0.968436873747495
Test F1 Score: 0.9779913989375159
